In [1]:
import datetime
import numpy as np
import pandas as pd

import pickle

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
INPUT_DATA_FOLDER = "3_isw_preprocessed"
REPORTS_DATA_FILE = "all_days_isw_parsed_v2.csv"

OUTPUT_FOLDER = "4_all_data_preprocessed"
ISW_OUTPUT_DATA_FILE = "all_isw.csv"
WEATHER_EVENTS_OUTPUT_DATA_FILE = "all_hourly_weather_events.csv"
WEATHER_REGIONS = 'weather_regions.csv'

MODEL_FOLDER = "model"

tfidf_transformer_model = "tfidf_transformer"
count_vectorizer_model = "count_vectorizer"

tfidf_transformer_version = "v1"
count_vectorizer_version = "v1"

In [4]:
def isNaN(num):
    return num != num

In [5]:
df_isw = pd.read_csv(f"{INPUT_DATA_FOLDER}/{REPORTS_DATA_FILE}", sep=";")


KeyboardInterrupt



In [ ]:
df_isw["date_datetime"] = pd.to_datetime(df_isw["date"])

In [ ]:
df_isw['date_tomorrow_datetime'] = df_isw['date_datetime'].apply(lambda x: x+datetime.timedelta(days=1))

In [ ]:
df_isw = df_isw.rename(columns = {"date_datetime":"report_date"})
df_isw.to_csv(f"{OUTPUT_FOLDER}/{ISW_OUTPUT_DATA_FILE}", sep=";", index=False)

In [ ]:
df_isw.head()

In [ ]:
df_isw = df_isw.drop(['short_url', 'title', 'full_url', 'main_html_v2', 'report_text_stemm', 'report_date', 'date_tomorrow_datetime', 'text_title', 'main_html'], axis = 1)

In [ ]:
df_isw.head()

# event data

In [ ]:
EVENTS_DATA_FOLDER = "5_events"
EVENTS_DATA_FILE = "all_events.csv"

In [ ]:
df_events = pd.read_csv(f"{EVENTS_DATA_FOLDER}/{EVENTS_DATA_FILE}", sep =";")

In [ ]:
# df_events.head()

In [ ]:
df_events["start_time"] = pd.to_datetime(df_events["start"])
df_events["end_time"] = pd.to_datetime(df_events["end"])

In [ ]:
df_events["start_hour"] = df_events['start_time'].dt.floor('H')
df_events["end_hour"] = df_events['end_time'].dt.ceil('H')

In [ ]:
df_events["start_hour"] = df_events.apply(lambda x: x["start_hour"] if not isNaN(x["start_hour"]) else x["event_hour"] , axis=1)
df_events["end_hour"] = df_events.apply(lambda x: x["end_hour"] if not isNaN(x["end_hour"]) else x["event_hour"] , axis=1)

In [ ]:
# df_events.tail()

In [ ]:
df_events.loc[:, 'all_region'].unique()

In [ ]:
df_events['date_datetime'] =0

In [ ]:
def date_time(df):
    divider = " "
    for i in range(df.shape[0]):
        if divider in df.iloc[i,4]:
            temp_str = df.iloc[i,4]
            temp_list = temp_str.split(divider)
            df.iloc[i,14] = temp_list[0]

In [ ]:
date_time(df_events)

In [ ]:
count =0
for i in range(df_events.shape[0]):
    if i != 19932:
        if df_events.loc[i, 'date_datetime'] == df_events.loc[i+1, 'date_datetime']:
            count += 1
        else:
            count += 1
            if count == 1:
                df_events.loc[i, 'count_for_day'] = 1
                count = 0
            else: 
                k = i - count + 1
                for y in range(count):
                    df_events.loc[k, 'count_for_day'] = count
                    k += 1
                    if y == count -1:
                        count = 0
    else:
        count += 1
        k = i - count + 1
        for y in range(count):
            df_events.loc[k, 'count_for_day'] = count
            k += 1
            if y == count -1:
                count = 0

In [ ]:
df_events.rename(columns={'count_for_day': 'amount_of_alarms_for_one_day'}, inplace=True)

In [ ]:
# df_events.head()

In [ ]:
from datetime import date
def date_intersection(t1, t2):
    t1start, t1end = t1[0], t1[1]
    t2start, t2end = t2[0], t2[1]

    if t1end < t2start: return False
    if t1end == t2start: return True
    if t1start == t2start: return True
    if t1start < t2start and t2start < t1end: return True
    if t1start > t2start and t1end < t2end: return True
    if t1start < t2start and t1end > t2end: return True
    if t1start < t2end and t1end > t2end: return True
    if t1start > t2start and t1start < t2end: return True
    if t1start == t2end: return True
    if t1end == t2end: return True 
    if t1start > t2end: return False

In [ ]:
def count_alarms_sameTime(df):
    count = 0
    j = 0
    while(j != df.shape[0]):
        temp_dateS = df.iloc[j,4]
        temp_dateE = df.iloc[j,5]
        work_mask = (df.loc[:,'date_datetime'] == df.iloc[j,14])
        work_df = df.loc[work_mask,:]
        region_id_dict = {
 '1' : 0,
 '2' : 0,
 '3' : 0,
 '4' : 0,
 '5' : 0,
 '6' : 0,
 '7' : 0,
 '8' : 0,
 '9' : 0,
 '10' : 0,
 '12' : 0,
 '13' : 0,
 '14' : 0,
 '15' : 0,
 '16' : 0,
 '17' : 0,
 '18' : 0,
 '19' : 0,
 '20' : 0,
 '21' : 0,
 '22' : 0,
 '23' : 0,
 '24' : 0
}
        for i in range (work_df.shape[0]):
            if i == j:
                continue
            anotherDateS = work_df.iloc[i,4]
            anotherDateE = work_df.iloc[i,5]
            if df.iloc[j,14].split('-')[1] != work_df.iloc[i,14].split('-')[1]:
                continue
            if date_intersection((temp_dateS,temp_dateE),(anotherDateS,anotherDateE)):
                if df.iloc[j,0] != work_df.iloc[i,0]:
                    temp_str_id = str(work_df.iloc[i,0])
                    if region_id_dict[temp_str_id] != 1:
                        count+=1
                        region_id_dict[temp_str_id] = 1 
        df.loc[j,'intersection_alarm_region_counts'] = count
        count = 0
        j+=1

In [ ]:
count_alarms_sameTime(df_events)

In [ ]:
df_events.loc[:, 'intersection_alarm_region_counts'] = df_events.loc[:, 'intersection_alarm_region_counts'] +1

In [ ]:
# df_events.head()

In [ ]:
from datetime import datetime

def count_amount_for_24_hours(df):
    for index in range(df.shape[0]):
        df.loc[index, 'last_24_hour_in_region']= index
    
    one_day = pd.Timedelta(days=1)
    zero_timedelta = pd.Timedelta(hours=0, minutes=0)
    
    for i in range(25):
        mask_region = df.loc[:, 'region_id'] == i
        df_region = df.loc[mask_region, :]
        df_region.sort_values(by='start_hour')
        
        for j in range(df_region.shape[0]):
            count = 0
            timedelta = df_region.iloc[j, 12] - df_region['start_hour']
            
            count = (timedelta <= one_day) & (timedelta >= zero_timedelta) 
            count = count.sum()
            
            index_num = df_region[df_region['last_24_hour_in_region'] == df_region.iloc[j, 17]].index
            df.loc[index_num,'last_24_hour_in_region']= count

In [ ]:
count_amount_for_24_hours(df_events)

In [ ]:
df_events['intersection_alarm_region_counts'].max()

In [ ]:
# df_events.head()

# weather

In [ ]:
WEATHER_DATA_FOLDER = "prepared_csv"
WEATHER_DATA_FILE = "all_hourly_weather_events.csv"

In [ ]:
df_weather = pd.read_csv(f"{WEATHER_DATA_FOLDER}/{WEATHER_DATA_FILE}", sep =";")
df_weather["day_datetime"] = pd.to_datetime(df_weather["day_datetime"])

In [ ]:
df_weather = df_weather.drop(index=df_weather.index[:24])

In [ ]:
df_weather.head(26)
df_weather.index = df_weather.index -24

In [ ]:
df_weather.head()

In [ ]:
df_weather = df_weather.sort_values(by='day_datetime')

In [ ]:
df_weather = df_weather.drop(index=df_weather.index[:528])

In [ ]:
df_weather.head()

In [ ]:
df_weather.index = range(df_weather.shape[0])

In [ ]:
df_weather.head()

In [ ]:
exclude = [
"day_feelslikemax",
"day_feelslikemin",
"day_description",
"city_address",
"city_timezone",
"day_feelslike",
"day_precipprob",
"day_snow",
"day_snowdepth",
"day_windgust",
"day_windspeed",
"day_winddir",
"day_pressure",
"day_cloudcover",
"day_visibility",
"day_severerisk",
"day_conditions",
"day_icon",
"day_source",
"day_preciptype",
"day_stations",
"hour_icon",
"hour_feelslike"
]

In [ ]:
df_weather = df_weather.drop(exclude, axis=1)

In [ ]:
df_weather.columns

In [ ]:
df_weather["city"] = df_weather["city_resolvedAddress"].apply(lambda x: x.split(",")[0])
df_weather["city"] = df_weather["city"].replace('Хмельницька область', "Хмельницький")

In [ ]:
df_weather.head(5)

# Merging

In [ ]:
df_regions = pd.read_csv(f"Homework3/raw_csv/regions.csv")

In [ ]:
df_regions.head()

In [ ]:
df_weather_reg = pd.merge(df_weather, df_regions, left_on="city",right_on="center_city_ua")

In [ ]:
df_weather_reg.head()

In [ ]:
df_weather_reg.to_csv(f"{OUTPUT_FOLDER}/{WEATHER_REGIONS}", sep=";", index=False)

In [ ]:
import calendar

In [ ]:
df_events["start_hour_datetimeEpoch"]= df_events['start_hour'].apply(lambda x: int(x.timestamp())  if not isNaN(x) else None)
df_events["end_hour_datetimeEpoch"] = df_events['end_hour'].apply(lambda x: int(x.timestamp())  if not isNaN(x) else None)

In [ ]:
df_events.head()

In [ ]:
events_dict = df_events.to_dict('records')
events_by_hour = []

In [ ]:
events_dict[0]

In [ ]:
for event in events_dict:
    for d in pd.date_range(start=event["start_hour"], end=event["end_hour"], freq='1H'):
        et = event.copy()
        et["hour_level_event_time"] = d
        events_by_hour.append(et)

In [ ]:
df_events_v1 = pd.DataFrame.from_dict(events_by_hour)

In [ ]:
df_events_v1["hour_level_event_datetimeEpoch"] = df_events_v1["hour_level_event_time"].apply(lambda x: int(x.timestamp()) if not isNaN(x) else None)

In [ ]:
df_events_v1.head()

In [ ]:
df_events_v2 = df_events_v1.copy().add_prefix('event_')

In [ ]:
df_events_v2.head()

In [ ]:
df_weather_v2 = df_weather_reg.merge(df_events_v2, 
                                     how="left", 
                                     left_on=["region_alt","hour_datetimeEpoch"],
                                     right_on=["event_region_title","event_hour_level_event_datetimeEpoch"])

In [ ]:
df_weather_v2.head()

In [ ]:
df_weather_v2['event_start_hour'].unique()

In [ ]:
df_weather_v2['event_start_hour'].describe()

In [ ]:
df_weather_v2.to_csv(f"{OUTPUT_FOLDER}/{WEATHER_EVENTS_OUTPUT_DATA_FILE}", sep=";", index=False)

In [ ]:
df_weather_v2.shape